In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用 Vertex AI 特征存储（遗留版）与 Pandas 数据框架

<table align="left">
    <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/feature_store/sdk-feature-store-pandas.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 
        在 Colab 中运行
    </a>
  </td>
    
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/feature_store/sdk-feature-store-pandas.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/feature_store/sdk-feature-store-pandas.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

注意: 这个笔记本已在以下环境中进行过测试:
- Python 版本 = 3.9

## 概述

本笔记本介绍了使用Vertex AI SDK支持Vertex AI特征存储（遗留）的Pandas。有关Vertex AI SDK和Vertex AI特征存储（遗留）原生支持的先决条件和介绍，请阅读此[Colab笔记本](https://colab.sandbox.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/feature_store/sdk-feature-store.ipynb)。

了解更多关于[Vertex AI特征存储](https://cloud.google.com/vertex-ai/docs/featurestore)。

### 目标

在这本笔记本中，您将学习如何使用 `Vertex AI Feature Store (Legacy)` 与 pandas Dataframe。

本教程使用以下 Google Cloud ML 服务和资源：

- Vertex AI Feature Store (Legacy)

执行的步骤包括：

- 创建 `Featurestore`、`EntityType` 和 `Feature` 资源。
- 将特征值从 Pandas DataFrame 导入到实体类型中。
- 从在线特征存储中读取实体特征值到 Pandas DataFrame。
- 将批处理提供的特征值从您的特征存储中导入到 Pandas DataFrame。

您还将学习 `Vertex AI Feature Store (Legacy)` 在以下场景中的作用：

- 使用更新后的特征值进行在线提供。
- 在训练时获取特征值的正确性。

### 数据集

本教程是Vertex AI特征存储（传统版）教程笔记本的一部分。它使用电影推荐数据集作为示例，演示了Vertex AI特征存储（传统版）的各种功能。原始任务是训练一个模型，以预测用户是否会观看一部电影，并在线提供该模型。

### 成本

本教程使用 Google Cloud 的计费组件：

- Vertex AI
- 云存储

了解 [Vertex AI 价格信息](https://cloud.google.com/vertex-ai/pricing)，以及 [云存储价格信息](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您预期的使用情况生成成本估算。

## 安装

安装以下所需的软件包以执行此笔记本。

In [ ]:
! pip install --quiet --upgrade google-cloud-aiplatform \
                                google-cloud-bigquery \
                                google-cloud-bigquery-storage \
                                avro \
                                pyarrow \
                                pandas \
                                fsspec \
                                gcsfs

###仅限协作：取消下面的单元格注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的 Google Cloud 项目

**以下步骤是必需的，无论您使用的是哪种笔记本环境。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得 $300 的免费信用，可用于计算/存储成本。

2. [确保为您的项目启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您是在本地运行此笔记本，则需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

#### 设置您的项目 ID

**如果您不知道您的项目 ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 认证您的谷歌云帐户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关指示操作。

**1. Vertex AI Workbench**
* 无需操作，因为您已经被验证。

**2. 本地JupyterLab实例，取消注释并运行:**

In [ ]:
# ! gcloud auth login

3. 协作，并取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

服务账户或其他
*请参阅如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上授予云存储权限给您的服务账户。

### 创建一个云存储桶

创建一个存储桶，用作Vertex AI和存储中间产物（如数据集）的暂存桶。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有如果您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 导入库

In [ ]:
import datetime

import pandas as pd
from avro.datafile import DataFileReader
from avro.io import DatumReader
from google.cloud import aiplatform

### 初始化 Python 的 Vertex AI SDK

为您的项目和地区初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## 创建特征存储

Vertex AI Feature Store（Legacy）作为您的ML特征的集中且有组织的存储库。您可以存储，提供和监控特征的某些方面，比如它们的分布和漂移。了解有关[Vertex AI Feature Store（Legacy）数据模型](https://cloud.google.com/vertex-ai/docs/featurestore/concepts)和[Vertex AI Feature Store（Legacy）的好处](https://cloud.google.com/vertex-ai/docs/featurestore/overview#benefits)更多信息。

要开始此教程，您可以使用Python的Vertex AI SDK创建一个特征存储。特征存储是您的特征及其值的顶层容器。为此，您可以使用[`Featurestore.create()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Featurestore)方法，它将返回一个LRO（[长时间运行的操作](https://google.aip.dev/151)）。LRO启动一个异步作业。对于其他API方法，如更新或删除特征存储，也会返回LRO。

在创建特征存储时，您需要传递以下参数：

- `featurestore_id`：特征存储的唯一名称或ID。
- `online_store_fixed_node_count`：在线服务资源的配置。节点的数量不会自动扩展，但可以通过在更新时提供不同的值来手动扩展。

In [ ]:
# set the id or name for the featurestore
featurestore_id = "movie_predictions_unique"  # @param {type:"string"}

# Create featurestore
movie_predictions_feature_store = aiplatform.Featurestore.create(
    featurestore_id=featurestore_id, online_store_fixed_node_count=1
)

创建实体类型

使用Vertex AI Feature Store（传统版本），您可以创建和管理特征存储、实体类型和特征。实体类型是语义相关特征的集合。您可以根据与您的用例相关的概念定义自己的实体类型。例如，电影服务可能具有电影和用户实体类型，它们将相关特征分组在一起，这些特征对应于电影或顾客。

了解更多有关[实体类型](https://cloud.google.com/vertex-ai/docs/featurestore/concepts#entity_type)的信息。

实体类型在`Featurestore`类中创建。在下面，为电影推荐数据集创建以下实体类型`users`和`movies`。

在创建实体类型时，您传递以下参数：

- `entity_type_id`：实体类型的唯一名称或ID。
- `description`：（可选）实体类型的描述。

In [ ]:
# Create users entity type
users_entity_type = movie_predictions_feature_store.create_entity_type(
    entity_type_id="users",
    description="Users entity",
)

# Create movies entity type
movies_entity_type = movie_predictions_feature_store.create_entity_type(
    entity_type_id="movies",
    description="Movies entity",
)

## 创建特征

特征是实体类型的可测量属性或属性。例如，电影实体类型具有诸如average_rating和title等特征，用于跟踪电影的各种属性。特征与实体类型相关联。

了解有关[特征](https://cloud.google.com/vertex-ai/docs/featurestore/concepts#feature)的更多信息。

使用以下方法将定义的特征添加到实体类型`users`和`movies`中。

### 使用*`create_feature`*方法添加特征

您提供以下参数来创建特征：

- `feature_id`：特征的资源名称或标识符。
- `value_type`：特征值的类型。 BOOL、BOOL_ARRAY、DOUBLE、DOUBLE_ARRAY、INT64、INT64_ARRAY、STRING、STRING_ARRAY、BYTES之一。
- `description`：特征的描述。

In [ ]:
# Create age feature
users_feature_age = users_entity_type.create_feature(
    feature_id="age",
    value_type="INT64",
    description="User age",
)

# Create gender feature
users_feature_gender = users_entity_type.create_feature(
    feature_id="gender",
    value_type="STRING",
    description="User gender",
)

# Create liked_genres feature
users_feature_liked_genres = users_entity_type.create_feature(
    feature_id="liked_genres",
    value_type="STRING_ARRAY",
    description="An array of genres this user liked",
)

### 使用 `batch_create_features` 方法添加特征

您还可以使用字典格式的配置映射一次性添加多个特征。为此，您可以使用 `batch_create_features` 方法。

在下面，您定义了创建 `title`、`genres` 和 `average_rating` 特征。

In [ ]:
# define the features
movies_feature_configs = {
    "title": {
        "value_type": "STRING",
        "description": "The title of the movie",
    },
    "genres": {
        "value_type": "STRING",
        "description": "The genre of the movie",
    },
    "average_rating": {
        "value_type": "DOUBLE",
        "description": "The average rating for the movie, range is [1.0-5.0]",
    },
}
# create the features
movie_features = movies_entity_type.batch_create_features(
    feature_configs=movies_feature_configs,
)

## 从数据框中将特征值导入实体类型

特征存储捕获了某个实体类型在特定时间点上所属特征的特征值。在将您的特征值导入特征存储后，您可以稍后从实体类型中`读取`（在线）或`批量提供`（离线）特征值。

在本节中，您将学习如何将特征值从[Pandas数据框](https://pandas.pydata.org/)导入到一个实体类型中。

了解更多关于[特征值](https://cloud.google.com/vertex-ai/docs/featurestore/concepts#feature_value)的信息。

注：您也可以从BigQuery或Google Cloud存储中导入特征值。

### 从源获取电影推荐数据

定义用户和电影的数据源，并将它们复制到本地的**.avro**文件中。

In [ ]:
# set the users file source
GCS_USERS_AVRO_URI = (
    "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/users.avro"
)
# set the movies file source
GCS_MOVIES_AVRO_URI = (
    "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movies.avro"
)
# set the local file names
USERS_AVRO_FN = "users.avro"
MOVIES_AVRO_FN = "movies.avro"
# copy the files using gsutil
! gsutil cp $GCS_USERS_AVRO_URI $USERS_AVRO_FN
! gsutil cp $GCS_MOVIES_AVRO_URI $MOVIES_AVRO_FN

### 从avro文件加载数据

从下载的avro文件中将用户和电影数据加载到Pandas数据框中。

In [ ]:
# Define a class for reading the avro data


class AvroReader:
    def __init__(self, data_file):
        self.avro_reader = DataFileReader(open(data_file, "rb"), DatumReader())

    def to_dataframe(self):
        records = [record for record in self.avro_reader]
        return pd.DataFrame.from_records(data=records)

In [ ]:
# Load users data from avro file
users_avro_reader = AvroReader(data_file=USERS_AVRO_FN)
users_source_df = users_avro_reader.to_dataframe()
users_source_df.head()

In [ ]:
# Load movies data from avro file
movies_avro_reader = AvroReader(data_file=MOVIES_AVRO_FN)
movies_source_df = movies_avro_reader.to_dataframe()
movies_source_df.head()

### 将特征值导入实体类型

从数据框加载特征值到`users`实体类型中。

您提供以下参数用于导入数据：

- `feature_ids`：要导入值的特征的ID列表。这些特征必须存在于目标实体类型中，否则请求将失败。
- `feature_time`：保存每个实体中所有特征值的特征时间戳的源列。它也可以是所有要导入的实体的单个特征时间戳。
- `df_source`：包含要导入的源数据的Pandas DataFrame。
- `entity_id_field`：保存实体ID的源列。

了解有关[`EntityType.ingest_from_df()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.EntityType#google_cloud_aiplatform_EntityType_ingest_from_df)方法的更多信息。

In [ ]:
# Import the data for users
users_entity_type.ingest_from_df(
    feature_ids=["age", "gender", "liked_genres"],
    feature_time="update_time",
    df_source=users_source_df,
    entity_id_field="user_id",
)

同样，加载`movies`实体类型的特征值。

In [ ]:
# Import the data for movies
movies_entity_type.ingest_from_df(
    feature_ids=["average_rating", "title", "genres"],
    feature_time="update_time",
    df_source=movies_source_df,
    entity_id_field="movie_id",
)

## 从特征存储库中的实体在线读取特征值

Vertex AI Feature Store（Legacy）包括在线服务，可以让您为小批量实体读取特征值。当您希望从实体或多个实体类型中读取选定特征的值时，这是非常有益的。

注意：实体是实体类型的一个实例。例如，movie_01 和 movie_02 是实体类型 movie 的实体。

了解更多关于[Vertex AI Feature Store（Legacy）中的在线服务](https://cloud.google.com/vertex-ai/docs/featurestore/serving-online)。

### 为用户读取特征值

使用必要的实体id从实体类型 `users` 调用 [`EntityType.read()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.EntityType#google_cloud_aiplatform_EntityType_read) 方法。

In [ ]:
# read the data to a dataframe
users_read_df = users_entity_type.read(
    entity_ids=["dave", "alice", "charlie", "bob", "eve"],
)
# dispaly the dataframe
users_read_df.head()

### 读取电影的特征值

使用实体类型`movies`中所需的实体id调用[`EntityType.read()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.EntityType#google_cloud_aiplatform_EntityType_read)方法。

In [ ]:
# read the data to a dataframe
movies_read_df = movies_entity_type.read(
    entity_ids=["movie_01", "movie_02", "movie_03", "movie_04"],
    feature_ids=["title", "genres", "average_rating"],
)
# display the dataframe
movies_read_df.head()

## 从特征存储中批量提供特征值

Vertex AI Feature Store（传统版）还可以批量提供特征值，以实现高吞吐量。批量提供通常用于训练模型或批量预测。

了解更多关于[在Vertex AI Feature Store（传统版）中进行批量提供](https://cloud.google.com/vertex-ai/docs/featurestore/serving-batch#batch_serving_inputs)。

在本节中，您将学习如何使用Vertex AI Feature Store（传统版）中的批量提供功能准备训练样本。

### 从来源文件中加载读取实例

定义来源文件路径，其中包含一些具有特征值的样本。在这里，您可以使用Pandas从数据集中的`movie_prediction.csv`文件加载一些特征值。这些数据在调用批量提供功能时充当读取实例。

In [ ]:
# set the gcs source for samples
GCS_READ_INSTANCES_CSV_URI = "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movie_prediction.csv"

在加载数据时，将“timestamp”列解析为日期时间字段。这是因为特征存储期望在批量服务时读取实例时有一个时间戳字段。

In [ ]:
# load the data using pandas
read_instances_df = pd.read_csv(GCS_READ_INSTANCES_CSV_URI, parse_dates=["timestamp"])
# display the dataframe
read_instances_df.head()

批量提取特征值功能

使用 `batch_serve_to_df` 方法将批量响应提供给数据框，并提供以下参数：

- `serving_feature_ids`：用户定义的字典，用于定义批量提取/读取的实体类型及其特征。字典的键是提取实体类型的id，值是每个实体类型中提取特征的id列表。

- `read_instances_df`：包含读取实例的pandas数据框。每个读取实例应至少包含一个读取时间戳和一个或多个标识相应实体类型实体的实体ID。每个输出实例包含请求实体的特征值，在读取时间点上被串联在一起。

    一个示例的 `read_instances_df` 可能是：

    ```
    pd.DataFrame( data=[ { 
            "my_entity_type_id_1": "my_entity_type_id_1_entity_1", 
            "my_entity_type_id_2": "my_entity_type_id_2_entity_1", 
            "timestamp": "2020-01-01T10:00:00.123Z" ], ) 
    ```
    一个示例的 `batch_serve_output_df` 可能是 

    ```
    pd.DataFrame( data=[ { 
            "my_entity_type_id_1": "my_entity_type_id_1_entity_1", 
            "my_entity_type_id_2": "my_entity_type_id_2_entity_1", 
            "foo": "feature_id_1_1_feature_value", 
            "feature_id_1_2": "feature_id_1_2_feature_value", 
            "feature_id_2_1": "feature_id_2_1_feature_value", 
            "bar": "feature_id_2_2_feature_value", 
            "timestamp": "2020-01-01T10:00:00.123Z" ], ) 
    ``` 
        

注意：调用 `batch_serve_to_df` 方法会自动在同一GCP项目中创建和删除一个临时的bigquery数据集，用作将来自Vertex AI Feature Store（传统版）的批量提取特征值存储到数据框的中间存储。

了解更多关于[从Vertex AI Feature Store（传统版）批量提取到数据框](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Featurestore#google_cloud_aiplatform_Featurestore_batch_serve_to_df).

In [ ]:
# call the batch serve method
movie_predictions_df = movie_predictions_feature_store.batch_serve_to_df(
    serving_feature_ids={
        "users": ["age", "gender", "liked_genres"],
        "movies": ["title", "average_rating", "genres"],
    },
    read_instances_df=read_instances_df,
)
# display the dataframe
movie_predictions_df.head()

## 读取最新的特征值

在Vertex AI特征存储（Legacy）中，您可以访问最新或最后可用的特征值，除非提供了特定的时间。

现在，您可以通过将新数据导入实体类型并从特征存储中读取来测试此功能。

### 导入更新后的特征值

现在，您可以通过运行以下单元格更新特征值。

**注意：** 为了比较，您可以尝试打印之前从实体类型中读取的特征值（位于`movies_read_df`变量中）。

In [ ]:
# Create a dataframe for the new data
update_movies_df = pd.DataFrame(
    data=[["movie_03", 4.3], ["movie_04", 4.8]],
    columns=["movie_id", "average_rating"],
)

# Import the new data from the dataframe
movies_entity_type.ingest_from_df(
    feature_ids=["average_rating"],
    feature_time=datetime.datetime.now(),  # provide the current timestamp
    df_source=update_movies_df,
    entity_id_field="movie_id",
)

获取最新的特征值

从实体类型中读取将为您提供最新导入的特征值。

运行下面的单元格应该获取所有请求的实体的最新值，包括您在上一个单元格中添加的 `movie_03` 和 `movie_04`。

In [ ]:
# read the feature values from the entity type
update_movies_read_df = movies_entity_type.read(
    entity_ids=["movie_01", "movie_02", "movie_03", "movie_04"],
    feature_ids=["title", "genres", "average_rating"],
)
# display the dataframe
update_movies_read_df.head()

## 瞬时正确性

Vertex AI Feature Store（已弃用）在[特定时刻](https://cloud.google.com/vertex-ai/docs/featurestore/serving-batch#example_point-in-time_lookup)捕获特征的特征值。如果过去的数据中存在缺失值，您可以使用批处理服务来进行补充。

### 检查缺失数据
请记住，上次导入的批量提供的响应中有一些缺失数据。

In [ ]:
# check the missing data
movie_predictions_df.isna().sum()

### 回填/修正点时数据

根据时间戳填充缺失数据。

注意：时间戳字段必须使用 RFC 3339 格式（例如 2012-07-30T10:43:17.123Z），或者在加载到 BigQuery 时应与 Timestamp 数据类型兼容。这是因为 Vertex AI Feature Store（Legacy）在批量提供服务时会将数据加载到临时的 BigQuery 表中作为中间步骤。了解更多关于[从 dataframe 加载数据到 BigQuery](https://cloud.google.com/bigquery/docs/samples/bigquery-load-table-dataframe)。

In [ ]:
# Impute the users data
backfill_users_df = pd.DataFrame(
    data=[["bob", 34, "Male", ["Drama"], "2020-02-13 09:35:15+00:00"]],
    columns=["user_id", "age", "gender", "liked_genres", "update_time"],
)
# convert the timefield to datetime64[ns] (with timezone info)
backfill_users_df["update_time"] = pd.to_datetime(backfill_users_df["update_time"])
# display the dataframe
backfill_users_df.head()

In [ ]:
# Impute the movies data
backfill_movies_df = pd.DataFrame(
    data=[["movie_04", 4.2, "The Dark Knight", "Action", "2020-02-13 09:35:15+00:00"]],
    columns=["movie_id", "average_rating", "title", "genres", "update_time"],
)
# convert the timefield to datetime64[ns] (with timezone info)
backfill_movies_df["update_time"] = pd.to_datetime(backfill_movies_df["update_time"])
# display the dataframe
backfill_movies_df.head()

### 导入填充/校正后的数据

将dataframe中的估算时间点数据导入featureImpostore中的实体类型。

In [ ]:
# Import the users data
users_entity_type.ingest_from_df(
    feature_ids=["age", "gender", "liked_genres"],
    feature_time="update_time",
    df_source=backfill_users_df,
    entity_id_field="user_id",
)

In [ ]:
# Import the users data
movies_entity_type.ingest_from_df(
    feature_ids=["average_rating", "title", "genres"],
    feature_time="update_time",
    df_source=backfill_movies_df,
    entity_id_field="movie_id",
)

提取最新数据
批量将导入的回填数据提供给数据框，以确保特征存储已更新。

In [ ]:
# batch serve the latest data to a dataframe
backfill_movie_predictions_df = movie_predictions_feature_store.batch_serve_to_df(
    serving_feature_ids={
        "users": ["age", "gender", "liked_genres"],
        "movies": ["title", "average_rating", "genres"],
    },
    read_instances_df=read_instances_df,
)
# display the dataframe
backfill_movie_predictions_df.head()

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源：

- Vertex AI特征存储（Legacy）
- Cloud Storage 存储桶（将`delete_bucket`设置为True）

In [ ]:
# Delete the featurestore
movie_predictions_feature_store.delete(force=True)

# remove the local users and movies avro files
! rm {USERS_AVRO_FN} {MOVIES_AVRO_FN}

# Delete Cloud Storage objects that were created
delete_bucket = True
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI